## Results with more realistic pulses
In this notebook, we make the following changes vs. original submission:

(1) 1 gigasample for pulse output instead of 20 gigasample

(2) `d = 3` to account for leakage

(3) aggressive pulse regularization coefficients to ensure that pulse is Gaussian envelope-like and has small first and second derivatives (`{'envelope': 5.0, 'dwdt': 0.001, 'd2wdt2': 0.00001}`)

---

NOTE: I am defining the gate time as the shortest `total_time` where the error gets below 10^-3 within 2000 iterations of QOC.

I determined this shortest time by binary searching. This notebook just shows the final `total_time` found for each gate.

---
## Setup

In [1]:
from quantum_optimal_control.helper_functions.grape_functions import transmon_gate
from quantum_optimal_control.main_grape.grape import Grape
from quantum_optimal_control.core import hamiltonian

In [2]:
import numpy as np
import os, sys, math
from datetime import datetime
sys.path.append('..'); import config

data_path = config.DATA_PATH
file_name = datetime.today().strftime('%h%d')

In [3]:
d = 3  # this is the number of energy levels to consider (i.e. d-level qudits)
max_iterations = 2000
decay =  max_iterations / 2
convergence = {'rate': 0.01, 'max_iterations': max_iterations,
               'conv_target':1e-3, 'learning_rate_decay':decay, 'min_grad': 1e-12}
reg_coeffs = {'envelope': 5.0, 'dwdt': 0.001, 'd2wdt2': 0.00001}

In [4]:
sys.path.append('..')
from fqc import uccsd, util, qaoa

----
## H Gate: 20 ns

In [44]:
N = 1
connected_qubit_pairs = []
H0 = hamiltonian.get_H0(N, d)
Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d, connected_qubit_pairs)
states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
maxA = hamiltonian.get_maxA(N, d, connected_qubit_pairs)
U = np.array([[1, 1], [1, -1]]) / math.sqrt(2)
U = transmon_gate(U, d)

In [54]:
total_time = 20
steps = int(total_time)
SS = Grape(H0, Hops, Hnames, U, total_time, steps, states_concerned_list, convergence,
                 reg_coeffs=reg_coeffs,
                 use_gpu=False, sparse_H=False, method='ADAM', maxA=maxA,
                 show_plots=False, file_name=file_name, data_path=data_path)

data saved at: /project/ftchong/qoc/pranav/00045_Jul01.h5
Using 4 Taylor terms and 8 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :7.99e-01; Runtime: 0.1s; Iterations = 0, grads =   1.039e+01, unitary_metric = 1.00001
Error = :2.56e-01; Runtime: 0.6s; Iterations = 100, grads =   8.910e-03, unitary_metric = 1.00002
Error = :3.27e-02; Runtime: 1.0s; Iterations = 200, grads =   8.969e-01, unitary_metric = 1.00004
Error = :1.89e-02; Runtime: 1.4s; Iterations = 300, grads =   6.390e-03, unitary_metric = 1.00001
Error = :1.24e-02; Runtime: 1.8s; Iterations = 400, grads =   5.962e-03, unitary_metric = 0.99997
Error = :6.84e-03; Runtime: 2.2s; Iterations = 500, grads =   3.389e-03, unitary_metric = 0.99999
Error = :5.52e-03; Runtime: 

---
## CNOT Gate: 45 ns

In [77]:
N = 2
connected_qubit_pairs = [(0, 1)]
H0 = hamiltonian.get_H0(N, d)
Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d, connected_qubit_pairs)
states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
maxA = hamiltonian.get_maxA(N, d, connected_qubit_pairs)
U = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])
U = transmon_gate(U, d)

In [80]:
total_time = 45
steps = int(total_time)
SS = Grape(H0, Hops, Hnames, U, total_time, steps, states_concerned_list, convergence,
                 reg_coeffs=reg_coeffs,
                 use_gpu=False, sparse_H=False, method='ADAM', maxA=maxA,
                 show_plots=False, file_name=file_name, data_path=data_path)

data saved at: /project/ftchong/qoc/pranav/00063_Jul01.h5
Using 4 Taylor terms and 10 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.35e-01; Runtime: 0.1s; Iterations = 0, grads =   9.286e+00, unitary_metric = 1.00044
Error = :2.78e-01; Runtime: 0.9s; Iterations = 100, grads =   2.355e+00, unitary_metric = 1.00063
Error = :1.22e-01; Runtime: 1.6s; Iterations = 200, grads =   4.403e+01, unitary_metric = 1.00033
Error = :2.76e-02; Runtime: 2.3s; Iterations = 300, grads =   1.531e-02, unitary_metric = 1.00042
Error = :1.51e-02; Runtime: 3.1s; Iterations = 400, grads =   1.222e-01, unitary_metric = 1.00045
Error = :7.08e-03; Runtime: 3.8s; Iterations = 500, grads =   4.592e-03, unitary_metric = 1.00034
Error = :3.77e-03; Runtime:

---
## SWAP Gate: 59 ns

In [83]:
N = 2
connected_qubit_pairs = [(0, 1)]
H0 = hamiltonian.get_H0(N, d)
Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d, connected_qubit_pairs)
states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
maxA = hamiltonian.get_maxA(N, d, connected_qubit_pairs)
U = np.array([[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]])
U = transmon_gate(U, d)

In [91]:
total_time = 59
steps = int(total_time)
SS = Grape(H0, Hops, Hnames, U, total_time, steps, states_concerned_list, convergence,
                 reg_coeffs=reg_coeffs,
                 use_gpu=False, sparse_H=False, method='ADAM', maxA=maxA,
                 show_plots=False, file_name=file_name, data_path=data_path)

data saved at: /project/ftchong/qoc/pranav/00072_Jul01.h5
Using 4 Taylor terms and 10 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.72e-01; Runtime: 0.1s; Iterations = 0, grads =   1.038e+00, unitary_metric = 1.00070
Error = :7.50e-01; Runtime: 1.0s; Iterations = 100, grads =   1.408e-03, unitary_metric = 1.00053
Error = :7.49e-01; Runtime: 1.8s; Iterations = 200, grads =   1.009e-03, unitary_metric = 1.00056
Error = :7.32e-02; Runtime: 2.6s; Iterations = 300, grads =   1.196e+02, unitary_metric = 1.00055
Error = :9.68e-04; Runtime: 3.2s; Iterations = 365, grads =   6.418e-02, unitary_metric = 1.00058
data saved at: /project/ftchong/qoc/pranav/00072_Jul01.h5


---
## RZ(Θ) Gate: 1 ns (at most)
As expected, theta=pi takes the longest to execute.

In [120]:
N = 1
connected_qubit_pairs = []
H0 = hamiltonian.get_H0(N, d)
Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d, connected_qubit_pairs)
states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
maxA = hamiltonian.get_maxA(N, d, connected_qubit_pairs)
theta = np.pi
U = np.array([[1, 0], [0, np.exp(1j * theta)]])
U = transmon_gate(U, d)

In [121]:
total_time = 1
steps = int(total_time)
SS = Grape(H0, Hops, Hnames, U, total_time, steps, states_concerned_list, convergence,
                 reg_coeffs=reg_coeffs,
                 use_gpu=False, sparse_H=False, method='ADAM', maxA=maxA,
                 show_plots=False, file_name=file_name, data_path=data_path)

data saved at: /project/ftchong/qoc/pranav/00096_Jul01.h5
Using 4 Taylor terms and 8 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :3.87e-01; Runtime: 0.0s; Iterations = 0, grads =   1.593e+01, unitary_metric = 0.99999
Error = :7.54e-04; Runtime: 0.2s; Iterations = 16, grads =   1.023e+00, unitary_metric = 0.99999
data saved at: /project/ftchong/qoc/pranav/00096_Jul01.h5


---
## RX(Θ) Gate: 31 ns (at most)
Also here, the longest time is for theta = pi, as expected.

In [123]:
N = 1
connected_qubit_pairs = []
H0 = hamiltonian.get_H0(N, d)
Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d, connected_qubit_pairs)
states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
maxA = hamiltonian.get_maxA(N, d, connected_qubit_pairs)
theta = np.pi
U = np.array([[np.cos(theta / 2.0), np.sin(theta / 2.0) * -1j], [np.sin(theta / 2.0) * -1j, np.cos(theta / 2.0)]])
U = transmon_gate(U, d)

In [134]:
total_time = 31
steps = int(total_time)
SS = Grape(H0, Hops, Hnames, U, total_time, steps, states_concerned_list, convergence,
                 reg_coeffs=reg_coeffs,
                 use_gpu=False, sparse_H=False, method='ADAM', maxA=maxA,
                 show_plots=False, file_name=file_name, data_path=data_path)

data saved at: /project/ftchong/qoc/pranav/00107_Jul01.h5
Using 4 Taylor terms and 8 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.02e-01; Runtime: 0.1s; Iterations = 0, grads =   9.605e+00, unitary_metric = 0.99995
Error = :1.88e-01; Runtime: 0.7s; Iterations = 100, grads =   3.746e-03, unitary_metric = 0.99999
Error = :1.88e-01; Runtime: 1.1s; Iterations = 200, grads =   1.238e-03, unitary_metric = 0.99994
Error = :1.87e-01; Runtime: 1.6s; Iterations = 300, grads =   8.394e-04, unitary_metric = 1.00000
Error = :1.86e-01; Runtime: 2.1s; Iterations = 400, grads =   6.155e-04, unitary_metric = 1.00001
Error = :1.85e-01; Runtime: 2.6s; Iterations = 500, grads =   4.230e-04, unitary_metric = 1.00003
Error = :1.85e-01; Runtime: 


---

# Gate vs. GRAPE times for VQE and QAOA
(with the more realistic pulses)


In [7]:
def binary_search_for_shortest_pulse_time(min_time, max_time, tolerance=1):
    """Search between [min_time, max_time] up to 1ns tolerance."""
    min_steps, max_steps = min_time, max_time
    while min_steps + tolerance < max_steps:  # just estimate to +- tolerance ns
        mid_steps = int((min_steps + max_steps) / 2)
        total_time = mid_steps
        print('\n\ntrying total_time: %s for unitary of size %s' % (str(total_time), str(U.shape)))
        SS = Grape(H0, Hops, Hnames, U, total_time, mid_steps, states_concerned_list, convergence,
                         reg_coeffs=reg_coeffs,
                         use_gpu=False, sparse_H=False, method='Adam', maxA=maxA,
                         show_plots=False, file_name=file_name, data_path=data_path, save=False)
        if SS.l < SS.conv.conv_target:  # if converged, search lower half
            max_steps = mid_steps
        else:
            min_steps = mid_steps

    return mid_steps

---

## VQE (for H2 molecule)

In [ ]:
max_iterations = 2000
decay =  max_iterations / 2
convergence = {'rate':0.01, 'max_iterations': max_iterations,
               'conv_target':1e-3, 'learning_rate_decay':decay, 'min_grad': 1e-12, 'update_step': 20}

In [152]:
circuit = uccsd.get_uccsd_circuit('H2')
U = util.circuitutil.get_unitary(circuit)
U = transmon_gate(U, d)

N = 2
connected_qubit_pairs = util.get_nearest_neighbor_coupling_list(1, 2, directed=False)
H0 = hamiltonian.get_H0(N, d)
Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d, connected_qubit_pairs)
states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
maxA = hamiltonian.get_maxA(N, d, connected_qubit_pairs)

In [5]:
# with Gates
print('Times reflect unoptimized and unmapped -> after optimization (time goes down) --> after mapping (time goes up)')
benchmark, layout = ('H2', (1, 2))
circuit = uccsd.get_uccsd_circuit(benchmark)
unoptimized_unmapped_time = np.round(util.circuitutil.get_max_pulse_time(circuit, more_realistic=True), 4)
circuit = util.circuitutil.optimize_circuit(circuit)
unmapped_time = np.round(util.circuitutil.get_max_pulse_time(circuit, more_realistic=True), 4)
coupling_list = util.circuitutil.get_nearest_neighbor_coupling_list(layout[0], layout[1])
circuit = util.circuitutil.optimize_circuit(circuit, coupling_list)
mapped_time = np.round(util.circuitutil.get_max_pulse_time(circuit, more_realistic=True), 4)
template = 'benchmark: %s. Times %s --> optimized = %s --> mapped = %s'
print(template % (benchmark, unoptimized_unmapped_time, unmapped_time, mapped_time))

Times reflect unoptimized and unmapped -> after optimization (time goes down) --> after mapping (time goes up)
benchmark: H2. Times 543.0 --> optimized = 420.0 --> mapped = 420.0


In [158]:
shortest_time = binary_search_for_shortest_pulse_time(1, 100, tolerance=3)



trying total_time: 50 for unitary of size (9, 9)
Using 4 Taylor terms and 10 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.52e-01; Runtime: 0.1s; Iterations = 0, grads =   3.553e+00, unitary_metric = 1.00092
Error = :7.73e-01; Runtime: 0.4s; Iterations = 20, grads =   2.507e+00, unitary_metric = 1.00100
Error = :5.18e-01; Runtime: 0.6s; Iterations = 40, grads =   9.160e+00, unitary_metric = 1.00095
Error = :3.25e-01; Runtime: 0.7s; Iterations = 60, grads =   5.374e+00, unitary_metric = 1.00093
Error = :1.95e-01; Runtime: 0.8s; Iterations = 80, grads =   1.647e+01, unitary_metric = 1.00093
Error = :8.78e-02; Runtime: 0.9s; Iterations = 100, grads =   5.859e+00, unitary_metric = 1.00082
Error = :3.90e-02; Runtime: 1.1s; Iter

Error = :5.10e-02; Runtime: 3.1s; Iterations = 540, grads =   1.252e-02, unitary_metric = 1.00028
Error = :5.03e-02; Runtime: 3.2s; Iterations = 560, grads =   1.331e-02, unitary_metric = 1.00011
Error = :4.93e-02; Runtime: 3.3s; Iterations = 580, grads =   1.499e-02, unitary_metric = 1.00015
Error = :4.82e-02; Runtime: 3.4s; Iterations = 600, grads =   4.671e-02, unitary_metric = 1.00025
Error = :4.71e-02; Runtime: 3.6s; Iterations = 620, grads =   4.320e-01, unitary_metric = 1.00000
Error = :4.58e-02; Runtime: 3.7s; Iterations = 640, grads =   3.635e-02, unitary_metric = 1.00012
Error = :4.67e-02; Runtime: 3.8s; Iterations = 660, grads =   2.373e-02, unitary_metric = 1.00014
Error = :5.00e-02; Runtime: 3.9s; Iterations = 680, grads =   2.225e-02, unitary_metric = 1.00024
Error = :5.94e-02; Runtime: 4.0s; Iterations = 700, grads =   2.326e-02, unitary_metric = 1.00022
Error = :7.95e-02; Runtime: 4.1s; Iterations = 720, grads =   2.376e-02, unitary_metric = 1.00011
Error = :9.47e-02; R

Error = :1.22e-01; Runtime: 1.0s; Iterations = 120, grads =   5.566e-02, unitary_metric = 1.00050
Error = :1.10e-01; Runtime: 1.1s; Iterations = 140, grads =   1.076e+00, unitary_metric = 1.00053
Error = :1.10e-01; Runtime: 1.2s; Iterations = 160, grads =   1.713e+01, unitary_metric = 1.00066
Error = :9.56e-02; Runtime: 1.3s; Iterations = 180, grads =   1.106e+00, unitary_metric = 1.00049
Error = :9.06e-02; Runtime: 1.4s; Iterations = 200, grads =   1.083e-01, unitary_metric = 1.00060
Error = :8.66e-02; Runtime: 1.6s; Iterations = 220, grads =   4.281e-02, unitary_metric = 1.00045
Error = :8.28e-02; Runtime: 1.7s; Iterations = 240, grads =   1.670e-02, unitary_metric = 1.00031
Error = :7.85e-02; Runtime: 1.8s; Iterations = 260, grads =   8.871e-03, unitary_metric = 1.00039
Error = :8.93e-02; Runtime: 1.9s; Iterations = 280, grads =   3.001e+01, unitary_metric = 1.00042
Error = :6.58e-02; Runtime: 2.0s; Iterations = 300, grads =   2.377e+00, unitary_metric = 1.00036
Error = :5.26e-02; R

Error = :3.75e-03; Runtime: 11.2s; Iterations = 1800, grads =   1.082e-03, unitary_metric = 1.00037
Error = :3.67e-03; Runtime: 11.4s; Iterations = 1820, grads =   1.072e-03, unitary_metric = 1.00035
Error = :3.74e-03; Runtime: 11.5s; Iterations = 1840, grads =   1.062e-03, unitary_metric = 1.00032
Error = :3.65e-03; Runtime: 11.7s; Iterations = 1860, grads =   1.048e-03, unitary_metric = 1.00039
Error = :3.57e-03; Runtime: 11.8s; Iterations = 1880, grads =   1.040e-03, unitary_metric = 1.00054
Error = :3.59e-03; Runtime: 11.9s; Iterations = 1900, grads =   1.038e-03, unitary_metric = 1.00050
Error = :3.49e-03; Runtime: 12.1s; Iterations = 1920, grads =   1.024e-03, unitary_metric = 1.00051
Error = :3.52e-03; Runtime: 12.2s; Iterations = 1940, grads =   1.010e-03, unitary_metric = 1.00043
Error = :3.51e-03; Runtime: 12.3s; Iterations = 1960, grads =   9.881e-04, unitary_metric = 1.00035
Error = :3.46e-03; Runtime: 12.4s; Iterations = 1980, grads =   9.875e-04, unitary_metric = 1.00042


Error = :4.29e-03; Runtime: 9.1s; Iterations = 1380, grads =   1.017e-03, unitary_metric = 1.00056
Error = :4.14e-03; Runtime: 9.2s; Iterations = 1400, grads =   9.896e-04, unitary_metric = 1.00049
Error = :4.21e-03; Runtime: 9.4s; Iterations = 1420, grads =   9.828e-04, unitary_metric = 1.00052
Error = :4.12e-03; Runtime: 9.5s; Iterations = 1440, grads =   9.386e-04, unitary_metric = 1.00071
Error = :4.06e-03; Runtime: 9.6s; Iterations = 1460, grads =   9.513e-04, unitary_metric = 1.00058
Error = :4.06e-03; Runtime: 9.8s; Iterations = 1480, grads =   9.305e-04, unitary_metric = 1.00053
Error = :3.87e-03; Runtime: 9.9s; Iterations = 1500, grads =   9.423e-04, unitary_metric = 1.00063
Error = :4.01e-03; Runtime: 10.1s; Iterations = 1520, grads =   9.395e-04, unitary_metric = 1.00036
Error = :4.06e-03; Runtime: 10.2s; Iterations = 1540, grads =   9.386e-04, unitary_metric = 1.00030
Error = :4.00e-03; Runtime: 10.3s; Iterations = 1560, grads =   9.352e-04, unitary_metric = 1.00031
Error =

Error = :3.87e-03; Runtime: 6.6s; Iterations = 960, grads =   2.398e-03, unitary_metric = 1.00051
Error = :3.74e-03; Runtime: 6.7s; Iterations = 980, grads =   2.479e-03, unitary_metric = 1.00049
Error = :3.85e-03; Runtime: 6.8s; Iterations = 1000, grads =   2.504e-03, unitary_metric = 1.00044
Error = :3.77e-03; Runtime: 7.0s; Iterations = 1020, grads =   2.572e-03, unitary_metric = 1.00041
Error = :3.66e-03; Runtime: 7.1s; Iterations = 1040, grads =   2.607e-03, unitary_metric = 1.00037
Error = :3.56e-03; Runtime: 7.2s; Iterations = 1060, grads =   2.675e-03, unitary_metric = 1.00049
Error = :3.45e-03; Runtime: 7.4s; Iterations = 1080, grads =   2.693e-03, unitary_metric = 1.00064
Error = :3.47e-03; Runtime: 7.5s; Iterations = 1100, grads =   2.714e-03, unitary_metric = 1.00057
Error = :3.59e-03; Runtime: 7.7s; Iterations = 1120, grads =   2.697e-03, unitary_metric = 1.00046
Error = :3.63e-03; Runtime: 7.8s; Iterations = 1140, grads =   2.647e-03, unitary_metric = 1.00029
Error = :3.6

Error = :8.70e-03; Runtime: 4.1s; Iterations = 520, grads =   1.074e+00, unitary_metric = 1.00052
Error = :7.76e-03; Runtime: 4.2s; Iterations = 540, grads =   1.583e-01, unitary_metric = 1.00041
Error = :6.87e-03; Runtime: 4.3s; Iterations = 560, grads =   6.152e-02, unitary_metric = 1.00075
Error = :6.40e-03; Runtime: 4.4s; Iterations = 580, grads =   9.796e-02, unitary_metric = 1.00086
Error = :6.44e-03; Runtime: 4.6s; Iterations = 600, grads =   4.474e-01, unitary_metric = 1.00061
Error = :5.88e-03; Runtime: 4.8s; Iterations = 620, grads =   2.462e-01, unitary_metric = 1.00064
Error = :5.71e-03; Runtime: 4.9s; Iterations = 640, grads =   2.800e-02, unitary_metric = 1.00043
Error = :5.69e-03; Runtime: 5.0s; Iterations = 660, grads =   5.650e-03, unitary_metric = 1.00053
Error = :5.51e-03; Runtime: 5.1s; Iterations = 680, grads =   2.424e-03, unitary_metric = 1.00050
Error = :5.22e-03; Runtime: 5.3s; Iterations = 700, grads =   2.318e-03, unitary_metric = 1.00070
Error = :5.26e-03; R

In [159]:
# with GRAPE:
print(shortest_time)

48


---

## QAOA (for N=3 node Erdos-Renyi graph with p=1)
(this is very small graph--purpose is just to capture circuit structure)

In [18]:
print('Times reflect unoptimized and unmapped -> after optimization (time goes down) --> after mapping (time goes up)')
for _ in range(5):
    N, p = 3, 1
    circuit = qaoa.get_qaoa_circuit(N, p, 'ErdosRenyi')
    unoptimized_unmapped_time = np.round(util.circuitutil.get_max_pulse_time(circuit, more_realistic=True), 4)
    circuit = util.circuitutil.optimize_circuit(circuit)
    unmapped_time = np.round(util.circuitutil.get_max_pulse_time(circuit, more_realistic=True), 4)
    coupling_list = util.circuitutil.get_nearest_neighbor_coupling_list(1, 3)
    circuit = util.circuitutil.optimize_circuit(circuit, coupling_list)
    mapped_time = np.round(util.circuitutil.get_max_pulse_time(circuit, more_realistic=True), 4)
    template = 'benchmark: %s. Times %s --> optimized = %s --> mapped = %s'
    print(template % (benchmark, unoptimized_unmapped_time, unmapped_time, mapped_time))

Times reflect unoptimized and unmapped -> after optimization (time goes down) --> after mapping (time goes up)
benchmark: H2. Times 233.0 --> optimized = 233.0 --> mapped = 351.0
benchmark: H2. Times 233.0 --> optimized = 233.0 --> mapped = 351.0
benchmark: H2. Times 233.0 --> optimized = 233.0 --> mapped = 292.0
benchmark: H2. Times 233.0 --> optimized = 233.0 --> mapped = 292.0
benchmark: H2. Times 142.0 --> optimized = 142.0 --> mapped = 142.0


In [20]:
sum([351, 351, 292, 292, 142]) / 5

285.6

In [8]:
full_times = []
N, p = 3, 1
max_iterations = 1000
decay = max_iterations / 2
for _ in range(5):
    circuit = qaoa.get_qaoa_circuit(N, p, 'ErdosRenyi')
    U = util.circuitutil.get_unitary(circuit)
    U = transmon_gate(U, d)
    connected_qubit_pairs = util.get_nearest_neighbor_coupling_list(1, 3, directed=False)
    H0 = hamiltonian.get_H0(N, d)
    Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d, connected_qubit_pairs)
    states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
    maxA = hamiltonian.get_maxA(N, d, connected_qubit_pairs)
    shortest_time = binary_search_for_shortest_pulse_time(10, 150, tolerance=3)
    full_times.append(shortest_time)



trying total_time: 80 for unitary of size (27, 27)
Using 4 Taylor terms and 11 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.99e-01; Runtime: 1.1s; Iterations = 0, grads =   4.109e-02, unitary_metric = 1.00298
Error = :3.05e-01; Runtime: 3.6s; Iterations = 100, grads =   4.698e+01, unitary_metric = 1.00277
Error = :1.08e-01; Runtime: 5.9s; Iterations = 200, grads =   5.144e+01, unitary_metric = 1.00310
Error = :6.03e-02; Runtime: 8.2s; Iterations = 300, grads =   2.468e+01, unitary_metric = 1.00332
Error = :3.95e-02; Runtime: 10.4s; Iterations = 400, grads =   9.871e+00, unitary_metric = 1.00315
Error = :3.11e-02; Runtime: 12.7s; Iterations = 500, grads =   3.793e+00, unitary_metric = 1.00312
Error = :2.35e-02; Runtime: 15

Error = :5.89e-03; Runtime: 22.8s; Iterations = 800, grads =   7.578e-02, unitary_metric = 1.00435
Error = :5.28e-03; Runtime: 25.6s; Iterations = 900, grads =   1.011e-03, unitary_metric = 1.00481
Error = :4.87e-03; Runtime: 28.4s; Iterations = 1000, grads =   8.716e-04, unitary_metric = 1.00430
Error = :4.44e-03; Runtime: 31.2s; Iterations = 1100, grads =   8.010e-04, unitary_metric = 1.00439
Error = :4.10e-03; Runtime: 34.0s; Iterations = 1200, grads =   6.458e-04, unitary_metric = 1.00441
Error = :4.09e-03; Runtime: 36.7s; Iterations = 1300, grads =   5.861e-04, unitary_metric = 1.00404
Error = :3.71e-03; Runtime: 39.5s; Iterations = 1400, grads =   5.306e-04, unitary_metric = 1.00397
Error = :3.53e-03; Runtime: 42.3s; Iterations = 1500, grads =   4.916e-04, unitary_metric = 1.00447
Error = :3.26e-03; Runtime: 45.1s; Iterations = 1600, grads =   4.567e-04, unitary_metric = 1.00444
Error = :3.05e-03; Runtime: 47.9s; Iterations = 1700, grads =   4.192e-04, unitary_metric = 1.00412
Er

Error = :2.19e-02; Runtime: 50.6s; Iterations = 1600, grads =   1.578e-03, unitary_metric = 1.00574
Error = :2.11e-02; Runtime: 53.7s; Iterations = 1700, grads =   1.535e-03, unitary_metric = 1.00576
Error = :2.03e-02; Runtime: 56.8s; Iterations = 1800, grads =   1.524e-03, unitary_metric = 1.00565
Error = :1.95e-02; Runtime: 59.9s; Iterations = 1900, grads =   1.429e-03, unitary_metric = 1.00551
Error = :1.92e-02; Runtime: 63.1s; Iterations = 2000, grads =   1.349e-03, unitary_metric = 1.00531


trying total_time: 132 for unitary of size (27, 27)
Using 4 Taylor terms and 11 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.91e-01; Runtime: 0.2s; Iterations = 0, grads =   1.025e+00, unitary_metric = 1.00460
Error = :9.96e-01; Ru

Error = :7.78e-02; Runtime: 8.3s; Iterations = 500, grads =   5.980e-02, unitary_metric = 1.00207
Error = :7.83e-02; Runtime: 9.9s; Iterations = 600, grads =   3.177e-02, unitary_metric = 1.00199
Error = :7.74e-02; Runtime: 11.5s; Iterations = 700, grads =   4.562e+00, unitary_metric = 1.00215
Error = :6.74e-02; Runtime: 13.1s; Iterations = 800, grads =   5.736e-02, unitary_metric = 1.00200
Error = :6.01e-02; Runtime: 14.7s; Iterations = 900, grads =   3.202e-03, unitary_metric = 1.00193
Error = :5.72e-02; Runtime: 16.3s; Iterations = 1000, grads =   2.703e-03, unitary_metric = 1.00197
Error = :5.29e-02; Runtime: 17.9s; Iterations = 1100, grads =   2.986e-03, unitary_metric = 1.00208
Error = :4.74e-02; Runtime: 19.5s; Iterations = 1200, grads =   3.499e-03, unitary_metric = 1.00152
Error = :4.71e-02; Runtime: 21.1s; Iterations = 1300, grads =   4.121e-03, unitary_metric = 1.00178
Error = :5.04e-02; Runtime: 22.8s; Iterations = 1400, grads =   4.704e-03, unitary_metric = 1.00171
Error =

Error = :6.65e-03; Runtime: 31.3s; Iterations = 1400, grads =   9.141e-02, unitary_metric = 1.00219
Error = :5.88e-03; Runtime: 33.5s; Iterations = 1500, grads =   2.150e-02, unitary_metric = 1.00248
Error = :6.90e-03; Runtime: 35.8s; Iterations = 1600, grads =   2.020e-02, unitary_metric = 1.00185
Error = :8.31e-03; Runtime: 38.0s; Iterations = 1700, grads =   2.589e-02, unitary_metric = 1.00193
Error = :8.23e-03; Runtime: 40.2s; Iterations = 1800, grads =   3.686e-02, unitary_metric = 1.00166
Error = :6.58e-03; Runtime: 42.4s; Iterations = 1900, grads =   1.631e-02, unitary_metric = 1.00215
Error = :5.69e-03; Runtime: 44.7s; Iterations = 2000, grads =   1.289e-02, unitary_metric = 1.00199


trying total_time: 77 for unitary of size (27, 27)
Using 4 Taylor terms and 11 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss in

Error = :6.61e-01; Runtime: 14.7s; Iterations = 400, grads =   1.276e+02, unitary_metric = 1.00526
Error = :2.33e-01; Runtime: 18.3s; Iterations = 500, grads =   5.483e+02, unitary_metric = 1.00487
Error = :2.88e-02; Runtime: 21.8s; Iterations = 600, grads =   2.205e-01, unitary_metric = 1.00526
Error = :3.48e-02; Runtime: 25.4s; Iterations = 700, grads =   6.757e+01, unitary_metric = 1.00552
Error = :1.39e-02; Runtime: 28.9s; Iterations = 800, grads =   3.157e+00, unitary_metric = 1.00537
Error = :7.09e-03; Runtime: 32.4s; Iterations = 900, grads =   3.943e-01, unitary_metric = 1.00560
Error = :4.67e-03; Runtime: 36.0s; Iterations = 1000, grads =   1.829e+00, unitary_metric = 1.00532
Error = :2.34e-03; Runtime: 39.6s; Iterations = 1100, grads =   7.349e-04, unitary_metric = 1.00575
Error = :1.44e-03; Runtime: 43.1s; Iterations = 1200, grads =   5.890e-04, unitary_metric = 1.00531
Error = :9.80e-04; Runtime: 44.1s; Iterations = 1224, grads =   6.775e-04, unitary_metric = 1.00540


tryi



trying total_time: 45 for unitary of size (27, 27)
Using 4 Taylor terms and 11 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.96e-01; Runtime: 0.1s; Iterations = 0, grads =   7.593e-02, unitary_metric = 1.00174
Error = :3.90e-02; Runtime: 1.9s; Iterations = 100, grads =   4.824e+01, unitary_metric = 1.00176
Error = :9.85e-03; Runtime: 3.5s; Iterations = 200, grads =   2.363e+00, unitary_metric = 1.00155
Error = :7.05e-03; Runtime: 5.2s; Iterations = 300, grads =   1.986e-03, unitary_metric = 1.00146
Error = :7.42e-03; Runtime: 6.8s; Iterations = 400, grads =   1.462e+00, unitary_metric = 1.00148
Error = :6.21e-03; Runtime: 8.4s; Iterations = 500, grads =   1.410e-03, unitary_metric = 1.00152
Error = :6.16e-03; Runtime: 10.0

Error = :7.71e-03; Runtime: 5.5s; Iterations = 300, grads =   9.870e-01, unitary_metric = 1.00188
Error = :6.98e-03; Runtime: 7.2s; Iterations = 400, grads =   1.852e+00, unitary_metric = 1.00227
Error = :5.88e-03; Runtime: 8.9s; Iterations = 500, grads =   7.323e-03, unitary_metric = 1.00173
Error = :5.52e-03; Runtime: 10.6s; Iterations = 600, grads =   9.182e-04, unitary_metric = 1.00178
Error = :5.17e-03; Runtime: 12.4s; Iterations = 700, grads =   8.337e-04, unitary_metric = 1.00191
Error = :4.56e-03; Runtime: 14.1s; Iterations = 800, grads =   6.616e-04, unitary_metric = 1.00148
Error = :4.22e-03; Runtime: 15.8s; Iterations = 900, grads =   5.606e-04, unitary_metric = 1.00141
Error = :3.91e-03; Runtime: 17.6s; Iterations = 1000, grads =   4.681e-04, unitary_metric = 1.00144
Error = :3.60e-03; Runtime: 19.3s; Iterations = 1100, grads =   3.807e-04, unitary_metric = 1.00168
Error = :3.61e-03; Runtime: 21.0s; Iterations = 1200, grads =   3.224e-04, unitary_metric = 1.00161
Error = :3

In [11]:
# with GRAPE:
print(full_times)
print('average: %s' % (sum(full_times) / len(full_times)))

[103, 129, 77, 117, 55]
average: 96.2


---

### QAOA N = 3, p = 1 with Original Submission Specs
The smallest QAOA graph we presented results for in the original submission was N=6. For a fair comparison to the N = 3, p = 1 case with realistic pulses, here are the results with the specs from the original specs.

In [9]:
# with Gates (setting more_realistic to False)
mapped_times = []
for _ in range(10):
    N, p = 3, 1
    circuit = qaoa.get_qaoa_circuit(N, p, 'ErdosRenyi')
    unoptimized_unmapped_time = np.round(util.circuitutil.get_max_pulse_time(circuit, more_realistic=False), 4)
    circuit = util.circuitutil.optimize_circuit(circuit)
    unmapped_time = np.round(util.circuitutil.get_max_pulse_time(circuit, more_realistic=False), 4)
    coupling_list = util.circuitutil.get_nearest_neighbor_coupling_list(1, 3)
    circuit = util.circuitutil.optimize_circuit(circuit, coupling_list)
    mapped_time = np.round(util.circuitutil.get_max_pulse_time(circuit, more_realistic=False), 4)
    mapped_times.append(mapped_time)
print(mapped_times)
print('average: %s' % (sum(mapped_times) / len(mapped_times)))

[11.9, 11.9, 19.9, 11.9, 19.9, 11.9, 11.9, 3.9, 42.7, 3.9]
average: 14.980000000000004


In [20]:
d = 2
reg_coeffs = {}

def binary_search_for_shortest_pulse_time(min_time, max_time, tolerance=1):
    """Search between [min_time, max_time] up to tolerance. Assumes 20 steps per ns.""" 
    min_steps, max_steps = min_time * 20, max_time * 20
    while min_steps + tolerance < max_steps: # just estimate to +- 1ns
        mid_steps = int((min_steps + max_steps) / 2)
        total_time = mid_steps / 20.0
        print('\n\ntrying total_time: %s for unitary of size %s' % (str(total_time), str(U.shape)))
        SS = Grape(H0, Hops, Hnames, U, total_time, mid_steps, states_concerned_list, convergence,
                    reg_coeffs=reg_coeffs,
                    use_gpu=False, sparse_H=False, method='ADAM', maxA=maxA, show_plots=False, file_name=file_name, data_path=data_path)
        if SS.l < SS.conv.conv_target: # if converged, search lower half 
            max_steps = mid_steps
        else:
            min_steps = mid_steps
    return mid_steps / 20

In [27]:
full_times = []
N, p = 3, 1
for _ in range(5):
    circuit = qaoa.get_qaoa_circuit(N, p, 'ErdosRenyi')
    U = util.circuitutil.get_unitary(circuit)
    U = transmon_gate(U, d)
    connected_qubit_pairs = [(0, 1), (1, 2), (2, 0)]
    H0 = hamiltonian.get_H0(N, d)
    Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d, connected_qubit_pairs)
    states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
    maxA = hamiltonian.get_maxA(N, d, connected_qubit_pairs)
    shortest_time = binary_search_for_shortest_pulse_time(0.5, 15, tolerance=3)
    full_times.append(shortest_time)



trying total_time: 7.75 for unitary of size (8, 8)
data saved at: /project/ftchong/qoc/pranav/00396_Jul01.h5
Using 4 Taylor terms and 3 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.90e-01; Runtime: 0.2s; Iterations = 0, grads =   9.749e-03, unitary_metric = 1.00002
Error = :4.99e-01; Runtime: 1.9s; Iterations = 100, grads =   1.804e-05, unitary_metric = 1.00000
Error = :1.82e-03; Runtime: 3.3s; Iterations = 200, grads =   7.620e-05, unitary_metric = 0.99998
Error = :9.37e-04; Runtime: 3.7s; Iterations = 208, grads =   1.839e-04, unitary_metric = 0.99999
data saved at: /project/ftchong/qoc/pranav/00396_Jul01.h5


trying total_time: 4.1 for unitary of size (8, 8)
data saved at: /project/ftchong/qoc/pranav/00397_Jul01.h5
Usi



trying total_time: 5.2 for unitary of size (8, 8)
data saved at: /project/ftchong/qoc/pranav/00401_Jul01.h5
Using 4 Taylor terms and 3 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.96e-01; Runtime: 0.2s; Iterations = 0, grads =   2.385e-03, unitary_metric = 1.00001
Error = :5.50e-01; Runtime: 1.4s; Iterations = 100, grads =   2.957e-04, unitary_metric = 0.99999
Error = :1.25e-01; Runtime: 2.5s; Iterations = 200, grads =   3.801e-04, unitary_metric = 0.99998
Error = :1.88e-02; Runtime: 3.5s; Iterations = 300, grads =   4.310e-05, unitary_metric = 0.99997
Error = :1.12e-02; Runtime: 4.6s; Iterations = 400, grads =   3.372e-06, unitary_metric = 0.99997
Error = :8.73e-03; Runtime: 5.6s; Iterations = 500, grads =   1.417e-06, u

Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.92e-01; Runtime: 0.2s; Iterations = 0, grads =   3.363e-03, unitary_metric = 1.00000
Error = :5.01e-01; Runtime: 1.4s; Iterations = 100, grads =   2.948e-05, unitary_metric = 1.00000
Error = :5.00e-01; Runtime: 2.5s; Iterations = 200, grads =   1.836e-07, unitary_metric = 1.00000
Error = :5.00e-01; Runtime: 3.5s; Iterations = 300, grads =   1.269e-07, unitary_metric = 1.00000
Error = :5.00e-01; Runtime: 4.6s; Iterations = 400, grads =   2.954e-07, unitary_metric = 1.00000
Error = :4.99e-01; Runtime: 5.7s; Iterations = 500, grads =   1.439e-06, unitary_metric = 1.00000
Error = :4.95e-01; Runtime: 6.8s; Iterations = 600, grads =   4.236e-05, unitary_metric = 0.99999
Error = :1.31e-01; Runtime: 7.9s; Iterations = 700, grads =   6.453e-04, unitary_metric = 0.99998
Error = :2.82e-02; Runtime: 8.9s; Iterations = 800, grads =   2.897e-05, unitary_metric = 0.99997
E

data saved at: /project/ftchong/qoc/pranav/00409_Jul01.h5


trying total_time: 7.75 for unitary of size (8, 8)
data saved at: /project/ftchong/qoc/pranav/00410_Jul01.h5
Using 4 Taylor terms and 3 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.89e-01; Runtime: 0.2s; Iterations = 0, grads =   6.007e-03, unitary_metric = 1.00002
Error = :9.04e-04; Runtime: 1.5s; Iterations = 77, grads =   2.981e-04, unitary_metric = 1.00000
data saved at: /project/ftchong/qoc/pranav/00410_Jul01.h5


trying total_time: 4.1 for unitary of size (8, 8)
data saved at: /project/ftchong/qoc/pranav/00411_Jul01.h5
Using 4 Taylor terms and 3 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propag

data saved at: /project/ftchong/qoc/pranav/00415_Jul01.h5


trying total_time: 2.6 for unitary of size (8, 8)
data saved at: /project/ftchong/qoc/pranav/00416_Jul01.h5
Using 4 Taylor terms and 3 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.99e-01; Runtime: 0.1s; Iterations = 0, grads =   1.619e-04, unitary_metric = 1.00000
Error = :2.50e-02; Runtime: 1.0s; Iterations = 100, grads =   1.418e-04, unitary_metric = 1.00000
Error = :3.09e-03; Runtime: 1.8s; Iterations = 200, grads =   6.222e-06, unitary_metric = 0.99999
Error = :9.93e-04; Runtime: 2.5s; Iterations = 299, grads =   1.180e-06, unitary_metric = 0.99999
data saved at: /project/ftchong/qoc/pranav/00416_Jul01.h5


trying total_time: 7.75 for unitary of size (8, 8)
dat

Error = :1.38e-03; Runtime: 12.8s; Iterations = 2000, grads =   1.316e-07, unitary_metric = 0.99999
data saved at: /project/ftchong/qoc/pranav/00421_Jul01.h5


trying total_time: 2.05 for unitary of size (8, 8)
data saved at: /project/ftchong/qoc/pranav/00422_Jul01.h5
Using 4 Taylor terms and 3 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :1.00e+00; Runtime: 0.1s; Iterations = 0, grads =   2.368e-07, unitary_metric = 1.00000
Error = :5.20e-02; Runtime: 0.9s; Iterations = 100, grads =   1.008e-04, unitary_metric = 0.99999
Error = :2.83e-02; Runtime: 1.6s; Iterations = 200, grads =   5.584e-05, unitary_metric = 1.00000
Error = :1.17e-02; Runtime: 2.2s; Iterations = 300, grads =   2.238e-05, unitary_metric = 0.99999
Error = :5.73

Error = :9.85e-04; Runtime: 1.3s; Iterations = 163, grads =   4.696e-06, unitary_metric = 1.00000
data saved at: /project/ftchong/qoc/pranav/00428_Jul01.h5


trying total_time: 1.6 for unitary of size (8, 8)
data saved at: /project/ftchong/qoc/pranav/00429_Jul01.h5
Using 4 Taylor terms and 3 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.75e-01; Runtime: 0.1s; Iterations = 0, grads =   4.934e-03, unitary_metric = 1.00000
Error = :2.79e-02; Runtime: 0.8s; Iterations = 100, grads =   2.394e-04, unitary_metric = 0.99999
Error = :4.30e-03; Runtime: 1.4s; Iterations = 200, grads =   1.239e-05, unitary_metric = 1.00000
Error = :1.40e-03; Runtime: 2.0s; Iterations = 300, grads =   2.401e-06, unitary_metric = 1.00000
Error = :1.00e-0

In [28]:
# with GRAPE
print(full_times)
print('Average: %s' % (sum(full_times) / len(full_times)))

[5.3, 5.3, 2.6, 1.95, 1.5]
Average: 3.3299999999999996
